# 5장 경사 하강법

#### 감사의 글

자료를 공개한 저자 오렐리앙 제롱과 강의자료를 지원한 한빛아카데미에게 진심어린 감사를 전합니다.

## 주요 내용

* 수학적으로 선형 회귀 모델 구하기

* 경사하강법으로 선형 회귀 모델 구하기

* 경사하강법 종류
    * 배치 경사하강법
    * 미니배치 경사하강법
    * 확률적 경사하강법(SGD)

## 선형 회귀

### 선형 회귀 모델 함수

- 한 개의 특성 $x_1$을 사용하는 $i$ 번째 훈련 샘플에 대한 예측값

$$\hat y^{(i)} = \theta_0 + \theta_1\, x_1^{(i)}$$

- $n\ge 1$ 개의 특성을 사용하는 $i$번째 훈련 샘플에 대한 예측값 
    - 예제: 캘리포니아 주택 가격 예측 모델

    $$\hat y^{(i)} = \theta_0 + \theta_1\, x_1^{(i)} + \cdots + \theta_{16}\, x_{16}^{(i)}$$
    ```
    ```
    * $\hat y^{(i)}$: $i$ 번째 훈련 샘플에 대한 예측값
    * $x_k^{(i)}$: $i$ 번째 훈련 샘플의 $k$ 번째 특성값
    * $\theta_k$: 편향($\theta_0$) 및 $k$ 번째 특성에 대한 가중치 파라미터

### 비용함수: 평균 제곱 오차(MSE)

* MSE를 활용한 선형 회귀 모델 성능 평가

$$
\mathrm{MSE}(\theta) := \mathrm{MSE}(\mathbf X, h_\theta) = 
\frac 1 m \sum_{i=1}^{m} \big(\theta^{T}\, \mathbf{x^{(i)}} - y^{(i)}\big)^2
$$

* 목표: $\mathrm{MSE}(\theta)$가 최소가 되도록 하는 $\theta$ 찾기

* 방식 1: 정규방정식 또는 특이값 분해(SVD) 활용
    * 드물지만 수학적으로 비용함수를 최소화하는 $\theta$ 값을 직접 계산할 수 있는 경우 활용
    * 계산복잡도가 $O(n^2)$ 이상인 행렬 연산을 수행해야 함. 
    * 따라서 특성 수($n$)이 큰 경우 메모리 관리 및 시간복잡도 문제때문에 비효율적임.

* 방식 2: 경사하강법
    * 특성 수가 매우 크거나 훈련 샘플이 너무 많아 메모리에 한꺼번에 담을 수 없을 때 적합
    * 일반적으로 선형 회귀 모델 훈련에 적용되는 기법

### 정규 방정식

정규 방정식을 이용하여 비용함수를 최소화 하는 $\theta$를 아래와 같이 구할 수 있음:

$$
\begin{align*}
\mathbf{X}\, \theta = \mathbf{y} 
& \quad\Rightarrow\quad ((\mathbf{X}^T)^{-1} \mathbf{X}^T)\, \mathbf{X}\, \theta = \mathbf{y} \\
& \quad\Rightarrow\quad (\mathbf{X}^T)^{-1}\, (\mathbf{X}^T\, \mathbf{X})\, \theta = \mathbf{y} \\
& \quad\Rightarrow\quad (\mathbf{X}^T\, \mathbf{X})\, \theta = \mathbf{X}^T\, \mathbf{y} \\
& \quad\Rightarrow\quad \theta = (\mathbf{X}^T\, \mathbf{X})^{-1}\, \mathbf{X}^T\, \mathbf{y}
\end{align*}
$$

#### SVD(특잇값 분해) 활용

* 행렬 연산과 역행렬 계산은 계산 복잡도가 $O(n^{2.4})$ 이상이고 항상 역행렬 계산이 가능한 것도 아님.

* 반면에, 특잇값 분해를 활용하여 얻어지는 무어-펜로즈(Moore-Penrose) 유사 역행렬 $\mathbf{X}^+$ 계산이 보다 효율적임.
    계산 복잡도는 $O(n^2)$.

$$
\hat \theta = 
\mathbf{X}^+\, \mathbf{y}
$$

## 경사 하강법

### 기본 아이디어

* 훈련 세트를 이용한 훈련 과정 중에 가중치 등과 같은 **파라미터를 조금씩 반복적으로 조정하기**

* 조정 기준: 비용 함수의 크기 줄이기

### 경사 하강법 관련 주요 개념

#### 최적 학습 모델

* 비용함수를 최소화하는 또는 효용함수를 최대화하는 파라미터를 사용하는 모델

#### 파라미터

* 예측값을 생성하는 함수로 구현되는 학습 모델에 사용되는 파라미터
* 예제: 선형 회귀 모델에 사용되는 편향과 가중치 파라미터 

$$\theta = \theta_0,\theta_1, \dots, \theta_n$$

#### 비용함수

* 모델이 얼마나 나쁜지를 계산해주는 함수
* 예제: 선형 회귀 모델의 평균 제곱 오차(MSE)

$$
\mathrm{MSE}(\theta) = 
\frac 1 m \sum_{i=1}^{m} \big(\theta^{T}\, \mathbf{x^{(i)}} - y^{(i)}\big)^2
$$

#### 전역 최솟값

* 비용함수가 가질 수 있는 최솟값
* 예제: 선형 회귀 모델의 평균 제곱 오차(MSE) 함수가 갖는 최솟값

#### 그레이디언트 벡터

* 다변수 함수의 미분값. 

* (그레이디언트) 벡터는 방향과 크기에 대한 정보 제공

* 그레이디언트가 가리키는 방향의 __반대 방향__으로 움직여야 가장 빠르게 전역 최솟값에 접근

* 예제: 선형 회귀 MSE의 그레이디언트 벡터 $\nabla_\theta \textrm{MSE}(\theta)$

$$
\nabla_\theta \textrm{MSE}(\theta) =
\begin{bmatrix}
    \frac{\partial}{\partial \theta_0} \textrm{MSE}(\theta) \\
    \frac{\partial}{\partial \theta_1} \textrm{MSE}(\theta) \\
    \vdots \\
    \frac{\partial}{\partial \theta_n} \textrm{MSE}(\theta)
\end{bmatrix} =
\frac{2}{m}\, \mathbf{X}^T\, (\mathbf{X}\, \theta^T - \mathbf y)
$$

#### 학습률

* 훈련 과정에서의 비용함수 파라미터 조정 비율

##### 예제: 선형회귀 모델 파라미터 조정 과정

* $\theta$를 임의의 값으로 지정한 후 훈련 시작

* 아래 단계를 $\theta$가 특정 값에 지정된 오차범위 내로 수렴할 때까지 반복
    1. (배치 크기로) 지정된 수의 훈련 샘플을 이용하여 학습.
    2. 학습 후 $\mathrm{MSE}(\theta)$ 계산.
    3. 이전 $\theta$에서 $\nabla_\theta \textrm{MSE}(\theta)$과 학습률 $\eta$를 곱한 값 빼기.

<div align="center"><img src="images/ch04/homl04-01.png" width="600"/></div>

$$\theta^{(\text{new})} = \theta^{(\text{old})}\, -\, \eta\cdot \nabla_\theta \textrm{MSE}(\theta^{(\text{old})})$$    

* 학습률이 너무 작은 경우: 비용 함수가 전역 최소값에 너무 느리게 수렴.

<div align="center"><img src="images/ch04/homl04-02.png" width="600"/></div>

* 학습률이 너무 큰 경우: 비용 함수가 수렴하지 않음.

<div align="center"><img src="images/ch04/homl04-03.png" width="600"/></div>

* (선형 회귀가 아닌 경우에) 시작점에 따라 지역 최솟값에 수렴하지 못할 수도 있음.

<div align="center"><img src="images/ch04/homl04-04.png" width="600"/></div>

* 선형 회귀와 학습률

    * 비용함수(MSE)가 볼록 함수. 즉, 지역 최솟값을 갖지 않음
    * 따라서 학습률이 너무 크지 않으면 언젠가는 전역 최솟값에 수렴

### 경사 하강법 종류

#### 배치 경사 하강법

* 전체 훈련 샘플을 대상으로 훈련한 후에, 즉 에포크마다 그레이디언트를 계산하여 파라미터 조정
* 훈련 세트가 크면 그레이디언트를 계산하는 데에 많은 시간 필요. 아주 많은 데이터를 저장해야 하는 메모리 문제도 발생 가능

#### 확률적 경사 하강법

* 배치 크기: 1
* 즉, 하나의 훈련 샘플을 학습할 때마다 그레이디언트를 계산해서 파라미터 조정

#### 미니배치 경사 하강법

* 배치 크기: 2에서 수백 사이
* 최적 배치 크기: 경우에 따라 다름. 여러 논문이 32 이하 추천

### 학습율과 경사 하강법의 관계

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml2/master/slides/images/ch04/homl04-04b.png" width="700"/></div>

### 확률적 경사 하강법

* 장점

    * 매우 큰 훈련 세트를 다룰 수 있음.
        예를 들어, 외부 메모리(out-of-core) 학습을 활용할 수 있음
    * 학습 과정이 매우 빠르며 파라미터 조정이 불안정 할 수 있기 때문에 지역 최솟값에 상대적으로 덜 민감

* 단점: 학습 과정에서 파라미터의 동요가 심해서 경우에 따라 전역 최솟값에 수렴하지 못하고 계속해서 발산할 가능성도 높음

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml2/master/slides/images/ch04/homl04-04c.png" width="400"/></div>

### 편향 vs 분산

* 편향(bias)
    - 실제로는 2차원 모델인데 1차원 모델을 사용하는 경우처럼 잘못된 가정으로 인해 발생.
    - 과소적합 발생 가능성 높음.

* 분산(variance)
    - 모델이 훈련 데이터에 민감하게 반응하는 정도
    - 고차 다항 회귀 모델의 경우 분산이 높아질 수 있음.
    - 과대적합 발생 가능성 높음.

* 편향과 분산의 트레이드 오프
    - 복잡한 모델일 수록 편향을 줄어들지만 분산을 커짐.